In [122]:
import random
class BlackJackEnv:

  rCount = 0
  roundStarted = False

  def __str__(self):
    return f"Deck: {self.deck} \nDiscard: {self.discard} \nRunning Count: {self.rCount} \nPlayer Hand: {self.playerHand} \nDealer Hand: {self.dealerHand} \nRound Started: {self.roundStarted}"

  def __init__(self, numDecks):
    self.deck = [2,3,4,5,6,7,8,9,10,10,10,10,11] * 4 * numDecks #ingoring suits
    self.rCount = 0 #Wont update for now, will just keep discrad pile and create method to update later
    self.discard = [] #Also contains cards that are in play (on the table)
    self.playerHand = []
    self.dealerHand = []
    
    self.shuffle()

  def shuffle(self):
    for card in self.discard:
      self.deck.append(card)
    self.discard = []
    #shuffle 5 times
    for i in range (5):
      random.shuffle(self.deck)

  def hand_value(self,hand):
    value = sum(hand)
    if value > 21 and 11 in hand:
      hand.remove(11)
      hand.append(1)
      value = sum(hand)
    return value
  

  #might need to change this account for aces that are already overdrawn for value 11
  def has_ace(self):
    return 11 in self.playerHand
  
  def deal_card(self):
    card = self.deck.pop()
    if card < 7:
      self.rCount += 1
    elif card > 9:
      self.rCount -= 1
    self.discard.append(card)
    return card
  
  def get_state(self):
    pl = self.hand_value(self.playerHand)
    dl = self.dealerHand[0]
    runningCount = self.rCount
    reward = self.get_reward()
    roundStarted = self.roundStarted
    #Returns: player hand-val, dealers first card, hasAce, roundstarted(bbool), reward
    return (pl,dl,self.has_ace(), roundStarted,reward,runningCount)
  
  def get_reward(self):
    if self.roundStarted:
      return 0
    else:

      playerValue = self.hand_value(self.playerHand)
      dealerValue = self.hand_value(self.dealerHand)
      if playerValue > 21:
        return -1
      if dealerValue > 21:
        return 1
      if playerValue > dealerValue:
        return 1
      if playerValue < dealerValue:
        return -1
      return 0
    
  def end_round(self):
    
    if self.get_reward() == 0:
      print("Tie")
    elif self.get_reward() == 1:
      print("Player wins")
    else:
      print("Dealer wins")
    
    self.playerHand = []
    self.dealerHand = []

  def start_round(self):
    if len(self.deck) < 10:
      self.shuffle()
    
    if self.roundStarted:
      print("Round already started")
      return None
    self.roundStarted = True
    self.playerHand = [self.deal_card(), self.deal_card()]
    self.dealerHand = [self.deal_card(), self.deal_card()]
    return self.get_state()

  def next(self,action):
    if not self.roundStarted:
      print("Round not started")
      return None
    
    if action == 1:
      self.playerHand.append(self.deal_card())
      if self.hand_value(self.playerHand) > 21:
        self.roundStarted = False
        state = self.get_state()
        self.end_round()
        return state
      return self.get_state()
    else:
      
      while self.hand_value(self.dealerHand) < 17:
        self.dealerHand.append(self.deal_card())
      self.roundStarted = False
      state = self.get_state()
      self.end_round()
      return state
    #Add option to double also
  
     



  
  
  
    











In [129]:
table = BlackJackEnv(1)
print(table)


Deck: [10, 11, 10, 6, 5, 2, 10, 4, 10, 3, 10, 10, 10, 5, 11, 9, 2, 6, 10, 10, 7, 5, 11, 9, 5, 10, 7, 10, 6, 10, 3, 10, 10, 8, 8, 4, 2, 10, 2, 8, 7, 6, 9, 10, 3, 4, 7, 3, 8, 11, 9, 4] 
Discard: [] 
Running Count: 0 
Player Hand: [] 
Dealer Hand: [] 
Round Started: False


In [144]:

table.start_round()


(16, 7, True, True, 0, 2)

In [148]:
table.next(1)

Dealer wins


(22, 7, False, False, -1, 2)

In [149]:
print(table)

Deck: [10, 11, 10, 6, 5, 2, 10, 4, 10, 3, 10, 10, 10, 5, 11, 9, 2] 
Discard: [4, 9, 11, 8, 3, 7, 4, 3, 10, 9, 6, 7, 8, 2, 10, 2, 4, 8, 8, 10, 10, 3, 10, 6, 10, 7, 10, 5, 9, 11, 5, 7, 10, 10, 6] 
Running Count: 2 
Player Hand: [] 
Dealer Hand: [] 
Round Started: False
